# Setting up a Dask Cluster on AzureML

In this lesson, we'll be using a dask cluster to replicate the [exercise we did in the Big Data section](exercises/Exercise_bigdata.ipynb) where we loaded global temperature data to measure global warming at a number of locations. You can get the data we're using for this [exercise here](https://www.dropbox.com/s/oq36w90hm9ltgvc/global_climate_data.zip?dl=0)). I'm also assuming you already have a Azure account and an AzureML workspace setup -- [go back here if you don't](cloud_azureml.ipynb)!

**Want to use Amazon AWS?** You can do that too! The package we use to launch our cluster below also supports AWS. We won't go through that here, but you can find [info on it here](https://cloudprovider.dask.org/en/stable/index.html).

If you want to follow along, just decompress the `ghcnd_daily.tar.gz` file and upload the resulting `.csv` into a Blob container using the web interface. Note this will take a little while. Will talk about more efficient methods of upload in our next tutorial.

To run this code, in addition the `dask` and `pandas`, which you should already have installed, you'll need to install the following packages (`azureml-sdk` and `dask_cloudprovider`) with the following commands:

```
conda install -c conda-forge azure-storage-blob # For managing storage
pip install azureml-sdk                         # For managing compute
pip install dask_cloudprovider==0.4.1
```

Note that `dask_cloudprovider` sometimes doesn't load the right version if you don't specify, and as of October 2020 the right version isn't even on `conda-forge`, so don't use `conda install`. You can also pip install `azure-storage-blob` if you prefer `pip` to `conda`. 

## Starting a Dask Cluster

In [1]:
%load_ext lab_black
from azureml.core import Workspace, Experiment
from dask_cloudprovider import AzureMLCluster

In [2]:
# I'm gonna load my subscription_id,
# resource_group, and workspace_name from a hidden file
# so y'all can't see them! You can just put in your code
# if its private.

import json

with open("/users/nick/azure_secrets/azure_config.json") as f:
    account = json.load(f)

In [3]:
# Register workspace with account info
ws = Workspace(
    subscription_id=account["subscription_id"],
    resource_group=account["resource_group"],
    workspace_name=account["workspace_name"],
)

In [ ]:
# Start up a cluster!
amlcluster = AzureMLCluster(
    ws,
    vm_size="STANDARD_DS3_V2",  # Azure VM size for the Compute Target,
    datastores=ws.datastores.values(),  # Azure ML Datastores to mount on the headnode
    environment_definition=ws.environments[
        "AzureML-Dask-CPU"
    ],  # Azure ML Environment to run on the cluster
    jupyter=True,  # Flag to start JupyterLab session on the headnode
    initial_node_count=4,  # number of nodes to start
    scheduler_idle_timeout=7200,  # scheduler idle timeout in seconds
)

In [ ]:
amlcluster

## Using Your Cluster

There are two ways to use your cluster: You can click on the link above to open a connection to JupyterLab running on one of the computers in your cluster, or connect from here with this command:

In [ ]:
from dask.distributed import Client

c = Client(amlcluster)

And you're off to the races! One note though -- if you decide to work from your own computer, you may get a warning about version differences between `dask` on the cloud on and on your own computer. I initially got:

```
/Users/Nick/miniconda3/lib/python3.7/site-packages/distributed/client.py:1130: VersionMismatchWarning: Mismatched versions found

+---------+---------------+---------------+---------------+
| Package | client        | scheduler     | workers       |
+---------+---------------+---------------+---------------+
| lz4     | None          | 3.1.0         | 3.1.0         |
| numpy   | 1.19.1        | 1.19.2        | 1.19.2        |
| python  | 3.7.8.final.0 | 3.6.9.final.0 | 3.6.9.final.0 |
+---------+---------------+---------------+---------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))
```

The `numpy` and `python` issues don't seem like to cause big problems (though if you have a mismatch and get problems down the road, consider changing your Python version!), but the fact that the schedulers and workers have one package I don't (`lz4`) is a problem, so I installed it before moving forward. (`lz4` is a compression algorithm used to send data back and forth, so not having it is a big problem).

First, we'll contect to the container where I put the climate data. As you saw in the last exercise, you can upload data using the Azure web interface, and I would do that if you want to do these exercises. However there are more efficient tools we'll cover in the [next lesson](cloud_azurestorage.ipynb).

In [ ]:
from azure.storage.blob import BlobServiceClient

# Load connection string so y'all can't see it!
with open("/users/nick/azure_secrets/azure_sa_connection_string.txt") as f:
    connection_string = f.read()

# Connect to storage account
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container = blob_service_client.get_container_client("globaltemps")

# Look at the files in the container for sanity check.
for f in container.list_blobs():
    print(f["name"] + "\n")

The step above was a fun way to see what's in my folders, but it's not actually required for using `dask` because `dask` can access Azure storage without the help of any other libraries -- you just need to be able to pass it your Storage Account name and Access Key, which you can find by going to your Azure Portal, then your Storage Account, and then clicking on "Access Keys" on the left menu. The syntax is:

```
import dask.dataframe as dd

storage_options={'account_name': ACCOUNT_NAME, 'account_key': ACCOUNT_KEY}

ddf = dd.read_csv('az://{CONTAINER}/{FOLDER}/*.csv', storage_options=storage_options)
ddf = dd.read_parquet('az://{CONTAINER}/folder.parquet', storage_options=storage_options)
```

But since I don't want you to see all my secret codes, I'm gonna load my information from a file. You can do this, but you can also put them in your code *if your code isn't public!*

In [ ]:
# Dask connects with a protocl
import json

with open("/users/nick/azure_secrets/azure_sa_name_and_key.json") as f:
    storage_options = json.load(f)

In [ ]:
import dask.dataframe as dd

# dask things the flag columns are floats, when really objects.
flag_dict = {}
for i in range(1, 32):
    for flag in ["q", "m"]:
        flag_dict.update({f"{flag}flag{i}": "object"})

temps = dd.read_csv(
    f"az://globaltemps/ghcnd_daily.csv",
    storage_options=storage_options,
    dtype=flag_dict,
)

In [ ]:
temps.head(10)

You will see that I added some code to explicitly name the types of some columns (the flags). As you may recall from our other lesson, dask isn't great at type inference, and will otherwise assume those are `float` columns instead of `objects`. Without that code, I get this error:

```
/azureml-envs/azureml_c6bd17107f471892400d23146f291775/lib/python3.6/site-packages/dask/dataframe/io/csv.py in coerce_dtypes()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+---------+--------+----------+
| Column  | Found  | Expected |
+---------+--------+----------+
| qflag1  | object | float64  |
| qflag10 | object | float64  |
| qflag11 | object | float64  |
| qflag12 | object | float64  |
| qflag13 | object | float64  |
| qflag14 | object | float64  |
| qflag15 | object | float64  |
| qflag16 | object | float64  |
| qflag17 | object | float64  |
| qflag18 | object | float64  |
| qflag19 | object | float64  |
| qflag2  | object | float64  |
| qflag20 | object | float64  |
```

Let's start by asking `dask` to go through this entire dataset and just pull out data for the station near my home in Colorado, and calculating the average daily max-temp for each month:

In [ ]:
temps = temps[temps["id"] == "USC00050848"]

Now as you may recall, `dask` hasn't actually run the code above -- it's just making a plan and waiting till I run `.compute()` to actually execute, so if I check on `temps`, I'll see:

In [ ]:
temps

Now since we'll most want to work with this data from one station, we can cache this subset dataframe with `c.persist(temps)` so `dask` won't have to re-load the original data next time we run `.compute()`. [More on caching here](https://docs.dask.org/en/latest/dataframe-best-practices.html#persist-intelligently), as well as other [dask best practices](https://docs.dask.org/en/latest/dataframe-best-practices.html#best-practices). Make sure to not just run `c.persists(temps)` but rather `temps = c.persist(temps)`.

Finally, one other note: this dataset is actually pretty small for Cloud computing, and since most of what we're doing is reading in data and filtering it, it involves a lot of moving data around. As a result it'd be much faster on a single really large VM (which doesn't need to use network connections to pass around data). But I wanted an example I could run relatively quickly and easily. :)

In [ ]:
temps = c.persist(temps)

OK, now we have our subset, and we can calculate what we want. Note that in reality, the data for this one station is actually easily small enough to put on my own computer, so I could have just run `data_on_own_comp = temps.compute()` above and moved the final dataset to my personal computer. Indeed, as you may recall from our [previous lesson on parallelism](parallelism.ipynb), if you don't have to use distributed computing, you probably don't want to! Even the authors of `dask` [are quick to remind users](https://docs.dask.org/en/latest/dataframe-best-practices.html#use-pandas): "For data that fits into RAM, Pandas can often be faster and easier to use than Dask DataFrame. While 'Big Data' tools can be exciting, they are almost always worse than normal data tools while those remain appropriate."

But let's carry on with this on the cluster for practice. 

In [ ]:
temps["value31"].value_counts().compute()

Now the really cool thing: if we use the link for our dashboard we got when we created our dask cluster, we can see our cluster working:

![azure_dask_scalingup](images/azure_dask_scalingup.png)


So let's replace those with missing, calculate averages across all the days in each month, and also convert from 1/10th of a degree Centigrade to Centigrade units:

In [ ]:
import numpy as np
import re

for i in range(1, 32):
    temps[f"value{i}"] = temps[f"value{i}"].replace(-9999, np.nan)

value_columns = [i for i in temps.columns if re.match("value.*", "value.*")]
temps["avg"] = temps[value_columns].mean(axis="columns")
temps["avg_C"] = temps["avg"] / 10

Now finally we can collect our results on our personal computer, create a "time" variable that uses years and months for plotting, and plot our temperatures!

In [ ]:
temps["time"] = temps["year"] + (temps["month"] - 1) / 12
temps = temps[["time", "avg_C"]]
df = temps.compute()

In [ ]:
from plotnine import *

(
    ggplot(df, aes(x="time", y="avg_C"))
    + geom_line()
    + geom_smooth(method="lowess", color="red")
    + ylab("Monthly Average of Daily Highs (C)")
    + xlab("Year")
    + ggtitle("Temperatures in Boulder, CO")
)

And that, my friends, is global warming. 

## Done with your cluster?

You have two options: you can shut it down manually with the method `.close()` on your cluster object. Thankfully, though, we also set a timeout condition when we created our cluster with the `scheduler_idle_timeout` keyword. We set it to 7200, so if your cluster is idle for two-hours, it will shut itself down (so you don't go broke). 

In [ ]:
amlcluster.close()

(For some reason when I get this I get the error above, but I can confirm on the Azure cite my cluster has been deleted.)

## Extensions and Fun Resources

- `dask-ml`: As a reminder, if you now want to do some machine learning, you can [use dask-ml on this system](https://ml.dask.org/), which does the same thing for `scikit-learn` that regular `dask` does for `pandas`. 
- Parallelize your own code with `delayed`: Just a reminder that while we've been using `dask` to emulate pandas in a distributed setting, it's also a framework you can use for distributing your own code! [Check out the `delayed`](https://docs.dask.org/en/latest/delayed.html) method to see how `dask` can manage your distributed workload. 
- Curious how `dask` compares to other tools for distributed computing? Here's a [conceptual comparison to Spark](https://docs.dask.org/en/latest/spark.html), and here's a [case study comparison of performance](https://arxiv.org/abs/1907.13030). Comparisons will usually depend a lot on the specifics of the work being done, but at least in this case, `dask` was a little faster than Spark. In the interview noted below, they also cite an example of `dask` beating Spark by 40x in a project they worked on. And here's [one report of a 2000x speed up doing random forests](https://www.saturncloud.io/s/random-forest-on-gpus-2000x-faster-than-apache-spark/) moving from Spark to dask. As they say, mileage may vary, but I think it's safe to say you aren't giving anything up performance wise by using `dask` and its familiar syntax instead of Spark.
- Interested in using `dask` in your company and want help? There's a great new company created by the founders of `dask` to provide enterprise support for `dask` [called coiled](https://coiled.io/) (No, I have no affiliation with them, I just think these companies that try to offer paid support services to businesses to help them move from closed source software to open source are a great way to help make open source software better). You can also hear a fun interview with the founders about [both `dask` and coiled here](https://talkpython.fm/episodes/show/285/dask-as-a-platform-service-with-coiled).
- The folks from coiled have also compiled a [great collection of videos and tutorials about dask and Python at scale here](https://coiled.io/videos/)
- Working with GPUs? There's a project to offer the kind of CPU parallelization we get from `dask` for GPUs called [dask-cudf](https://docs.rapids.ai/api/cudf/stable/dask-cudf.html) (part of the [RAPIDS project](https://rapids.ai/index.html). The project is young but growing quickly. My guess, though, is that those libraries will become the infrastructure for updates to tools like `dask-ml` rather than something most applied people need to play with. But putting it here as an FYI!

## Next Steps

OK, that's our short demo of computing on Azure with dask. I hope you found it useful!

As for next steps, I'd suggest the following readings, after which you should go play with your own project!

- [Managing Storage](cloud_azurestorage.ipynb): A discussion of better tools for uploading, downloading, and synchronizing data.
- [Mounting Storage](cloud_mountingazure.ipynb): A way to set-up easier access to your storage from your compute nodes.